In [3]:
import pandas as pd

In [8]:
df = pd.read_parquet("./data/all/EDV_P9931_PLC.BD.parquet")

In [9]:
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('.parquet'):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read each Parquet file into a DataFrame and append it to the list
        df = pd.read_parquet(file_path)
        dfs.append(df)

# Concatenate all DataFrames in the list into one
combined_df = pd.concat(dfs, ignore_index=True)

,historianTagnummer,hstWaarde,datumBeginMeting,datumEindeMeting
0,EDV_P9931_PLC.BD,14759.27,2021-01-01 00:00:00,2021-01-01 00:01:00
1,EDV_P9931_PLC.BD,14759.27,2021-01-01 00:01:00,2021-01-01 00:02:00
2,EDV_P9931_PLC.BD,14759.28,2021-01-01 00:02:00,2021-01-01 00:03:00
3,EDV_P9931_PLC.BD,14759.28,2021-01-01 00:03:00,2021-01-01 00:04:00
4,EDV_P9931_PLC.BD,14759.29,2021-01-01 00:04:00,2021-01-01 00:05:00
...,...,...,...,...
44635,EDV_P9931_PLC.BD,16646.21,2021-12-31 23:55:00,2021-12-31 23:56:00
44636,EDV_P9931_PLC.BD,16646.22,2021-12-31 23:56:00,2021-12-31 23:57:00
44637,EDV_P9931_PLC.BD,16646.22,2021-12-31 23:57:00,2021-12-31 23:58:00
44638,EDV_P9931_PLC.BD,16646.23,2021-12-31 23:58:00,2021-12-31 23:59:00
